In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import copy
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import time
from tqdm import tqdm

In [ ]:
MODEL_PATH = './models'

In [ ]:
model = torch.load(f'{MODEL_PATH}/model1.pt')

In [ ]:
test_df = pd.read_csv('test.csv')

In [ ]:
def clean(data):
    le = LabelEncoder()

    # Replace str by int using LabelEncoder
    data_le = copy.deepcopy(data)
    cols = ['attribute_0', 'attribute_1', 'product_code']
    for col in cols:
        data_le[col] = le.fit_transform(data[col])

    data_le = data_le.drop(['id', 'product_code'], axis=1)

    # filled missing values
    imputer = SimpleImputer(strategy='mean')
    final_data = pd.DataFrame(imputer.fit_transform(data_le))

    final_data.columns = data_le.columns

    return final_data

In [ ]:
test_df_clean = clean(test_df)

test_df_clean.head(5)

In [ ]:
test_data = test_df_clean.to_numpy()
test_ds = TaskDataset(test_data, return_y=False)
print("test num: ", test_ds.__len__())
test_dl = DataLoader(
    test_ds,
    batch_size=1,
    num_workers=0,
    drop_last=False,
    shuffle=False)

model.eval()
pred = []
for x in tqdm(test_dl):
    x = x.to(device)
    y_pred = model(x)
    output = torch.sigmoid(y_pred)
    output = output.cpu().detach().numpy()
    for i in range(len(output)):
        pred.append(output[i][0])
result = pd.DataFrame({'id': test_df['id'], 'failure': pred})
result.to_csv('submission.csv', index=0)
result